# (MBTI 별 Data 모으기: 생략) MBTI file 불러오기

In [17]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

In [18]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# Tokenize and Stemming


In [19]:
X_data = mbti['posts'].ravel()
y_data = mbti['type'].ravel()

print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [20]:
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
ps=PorterStemmer()

stop_words = set(stopwords.words('english'))

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        if word not in stop_words:
            tempData.append(ps.stem(word))
    stemData.append(tempData)
print("stemming 완료")
stemData = np.array(stemData)

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]

stemming 시작
stemming 완료


In [22]:
print(stemData[0])
vocab_size = len(np.unique(stemData))
print(vocab_size)

['oi', 'went', 'break', 'month', 'ago', 'togeth', 'year', 'plan', 'life', 'around', 'relationship', 'wasnt', 'one', 'break', 'relationship', 'might', 'imagin', 'ourenfj', 'pun', 'mani', 'punswel', 'person', 'dont', 'go', 'much', 'attract', 'gener', 'see', 'chang', 'that', 'good', 'alreadi', 'may', 'ask', 'want', 'mere', 'sexualsorri', 'infp', 'im', 'realli', 'postrock', 'post', 'go', 'leav', 'sneak', 'thread', 'dthat', 'doesnt', 'sound', 'enfj', 'think', 'x', 'id', 'never', 'act', 'cold', 'toward', 'romant', 'interest', 'get', 'warm', 'side', 'think', 'like', 'someon', 'way', 'want', 'straight', 'themnot', 'realli', 'im', 'mostli', 'guitar', 'player', 'bass', 'obsess', 'im', 'person', 'ask', 'bass', 'louder', 'least', 'relev', 'like', 'tri', 'new', 'instrument', 'sometim', 'add', 'aguitar', 'mostli', 'add', 'bass', 'glockenspiel', 'harmonica', 'drum', 'ocarinaahahah', 'im', 'sorri', 'im', 'laugh', 'hard', 'realli', 'plagu', 'never', 'thought', 'possiblemostli', 'sound', 'lyric', 'ill',

# Garbage Collection 1

In [23]:
import gc

'''del [[mbti]]

del sentence

del [[dataCombine]]

del [condition]

for ownsentence in allData:
    del ownsentence
del allData

for tokenData in tokenizedData:   # delete 'stemData'
    for data in tokenData:
        del data
    del tokenData
del tokenizedData'''

for clean_sentence in normalized_text:
    del clean_sentence
del normalized_text

del X_data

del tokenData
del tempData

del sentence

gc.collect()

88324

# Vectorization (tfidf)

In [24]:
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
X = tfidf.transform(stemData).toarray()
print(X)
print(X.shape)

y = np.array(y_data)
print(y)
print(y.shape)

[[0.        0.        0.        ... 0.        0.        0.       ]
 [0.0518922 0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]
(8675, 271354)
['ENFJ' 'ENFJ' 'ENFJ' ... 'ISTP' 'ISTP' 'ISTP']
(8675,)


In [25]:
b_Pers = {'I':0, 'E':1, 'N':0, 'S':1, 'F':0, 'T':1, 'J':0, 'P':1}
b_Pers_list = [{0:'I', 1:'E'}, {0:'N', 1:'S'}, {0:'F', 1:'T'}, {0:'J', 1:'P'}]

def translate_personality(personality):
    # transform mbti to binary vector
    
    return [b_Pers[l] for l in personality]

def translate_back(personality):
    # transform binary vector to mbti personality
    
    s = ""
    for i, l in enumerate(personality):
        s += b_Pers_list[i][l]
    return s
y = np.array([translate_personality(p) for p in y])
print(y)

[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ...
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]]


In [26]:
'''print(X.shape[0])
print(len(X[0].toarray()[0]))'''

'print(X.shape[0])\nprint(len(X[0].toarray()[0]))'

In [27]:
'''print(tfidf_matrix.shape)

max_words = max(len(l) for l in X)   # post의 최대 길이(단어 수)
print(max_words)
print('post의 평균 길이 :{}'.format(sum(map(len, stemData))/len(stemData)))
print('post의 전체 길이 :{}'.format(sum(map(len, stemData))))
post_num = len(stemData)    #post의 갯수
print(post_num)'''

"print(tfidf_matrix.shape)\n\nmax_words = max(len(l) for l in X)   # post의 최대 길이(단어 수)\nprint(max_words)\nprint('post의 평균 길이 :{}'.format(sum(map(len, stemData))/len(stemData)))\nprint('post의 전체 길이 :{}'.format(sum(map(len, stemData))))\npost_num = len(stemData)    #post의 갯수\nprint(post_num)"

# Garbage Collection

In [28]:
import psutil
# import os
# import gc

In [29]:
for data in stemData:   # delete 'stemData'
    for component in data:
        del component
    del data
del stemData

gc.collect()

110

In [30]:
import tensorflow as tf
from keras.utils import np_utils

from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)

print(X_train.shape)
print(X_train)
print(y_train.shape)
print(y_train)

model = Sequential()
model.add(Embedding(X.shape[0], 128))
model.add(LSTM(128))
model.add(Dense(16, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=5, batch_size=32)

(6072, 271354)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(6072, 4)
[[1 0 1 0]
 [0 1 1 0]
 [1 0 1 1]
 ...
 [0 0 1 1]
 [0 0 1 1]
 [1 0 1 1]]
Epoch 1/5


In [ ]:
model.save('.\\models\\LSTM_undersample.h5')

In [ ]:
model.summary()

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['acc'])
plt.plot(epochs, history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=32)